In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
import sklearn.tree
import sklearn.ensemble
from sklearn.cross_validation import cross_val_score
%matplotlib inline

In [24]:
data_train_read = pd.read_csv("train.csv")
data_test_read = pd.read_csv("test.csv")

In [29]:
data_train = data_train_read.copy()

In [30]:
data_test = data_test_read.copy()

In [15]:
table = data_train.corr()

In [16]:
#print table.head(12)
#print data.columns[[10, 11]]

tv = table.values
edges = []
for i in range(len(tv)):
    for j in range(i + 1, len(tv) - 1):
        if np.abs(tv[i][j]) > .97:
            edges.append([i, j])

print len(edges)

202


In [17]:
vertices = {}
for e in edges:
    vertices[e[0]] = True
    vertices[e[1]] = True
    
vertices = vertices.keys()
do_not_del = [False] * len(vertices)
power = [0] * len(vertices)
to_del = []
for edge in edges:
    power[vertices.index(edge[0])] += 1
    power[vertices.index(edge[1])] += 1
for i in range(len(vertices)):
    if power[i] == 1:
        to_del.append(vertices[i])
        
to_del = data_train.columns[to_del]

In [31]:
for cur in to_del:
    del data_train[cur]
    del data_test[cur]

In [19]:
X_train = data_train[data_train.columns[1:len(data_train.columns) - 1]]
y_train = data_train['TARGET']

In [47]:
clf = xgb.XGBRegressor(nthread=4, 
                       max_depth=8, 
                       n_estimators=30, 
                       subsample=0.5, 
                       learning_rate=0.006, 
                       seed=1337)

In [48]:
scores = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=3)
print scores.mean()

0.828266253987


In [35]:
clf.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.006, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=4,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1337, silent=True, subsample=0.5)

In [36]:
X_test = data_test[data_test.columns[1:]]

In [37]:
y_test = clf.predict(X_test)

In [40]:
trash_count = 0
for i in range(len(y_test)):
    if y_test[i] < 0:
        y_test[i] = 0
        trash_count += 1
    if y_test[i] > 1:
        y_test[i] = 1
        trash_count += 1
print trash_count
ans = pd.DataFrame(index=data_test['ID'], data={'TARGET': y_test})
ans.to_csv('kek.csv')

0


In [ ]:
from sklearn.preprocessing import normalize
